In [79]:

import os

import cv2
import numpy as np
from sklearn import cluster
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

from matplotlib import pyplot as plt

In [3]:
def gabor_fn(sigma, theta, Lambda, psi, gamma):
    sigma_x = sigma
    sigma_y = float(sigma) / gamma

    # Bounding box
    nstds = 3 # Number of standard deviation sigma
    xmax = max(abs(nstds * sigma_x * np.cos(theta)), abs(nstds * sigma_y * np.sin(theta)))
    xmax = np.ceil(max(1, xmax))
    ymax = max(abs(nstds * sigma_x * np.sin(theta)), abs(nstds * sigma_y * np.cos(theta)))
    ymax = np.ceil(max(1, ymax))
    xmin = -xmax
    ymin = -ymax
    (y, x) = np.meshgrid(np.arange(ymin, ymax + 1), np.arange(xmin, xmax + 1))

    # Rotation 
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)

    gb = np.exp(-.5 * (x_theta ** 2 / sigma_x ** 2 + y_theta ** 2 / sigma_y ** 2)) * np.cos(2 * np.pi / Lambda * x_theta + psi)
    return gb

def scale_minmax(img):
    return (img - np.min(img))/(np.max(img) - np.min(img))

In [4]:
sample_percent = 0.1
sample_size = 5

# Path leading to all cloud data
path = "C:\\Users\\peter\\Documents\\Cloud Project\\Data\\swimcat\\"
cloud_types = ["A-sky","B-pattern","C-thick-dark","D-thick-white","E-veil"]

# initializing dict to store randomly n selected training images 
train_set = {}

# Iterating over cloud types 
for ctype in cloud_types:
    fin = path + ctype +"\\images\\"
    
    # Reading all images of cloud type ctype from path fin + ctype
    allimgs = os.listdir(fin)
    
    # total number of images
    n_imgs = len(allimgs)
    
    # number of images to select
    n_sample = int(round(n_imgs*sample_percent,1))
    
    # shuffling list of images, slicing to get n images
    samples = allimgs[:sample_size]
    
    # Note that np.random.shuffle changes in place (ugh)
    np.random.shuffle(samples)
    
    # Dict key set to -> ctype, dict value -> samples
    train_set[ctype] = samples.copy()
    

In [49]:
from datetime import datetime

sparams = ((2,1),(4,1),(4,2),(6,1),(6,2),(6,3),(8,1),
            (8,2),(8,3),(10,1),(10,2),(10,3),(10,4))

gabor_keys = list(range(0,len(sparams)))
response_dict = {}

start = datetime.now()

# Iterating over: cloud types, images of type, filter type
# Temporarily stores responses in dict
for ctype in cloud_types:    
    for image in train_set[ctype]:
        for key, param in zip(gabor_keys, sparams):
#             print(key, param)
            sfilter = gabor_fn(param[0],0,param[1],0,1)

            imgpath = path + ctype + "\\images\\" + image

            img = scale_minmax(cv2.imread(imgpath,1))[::,::,0]
            output = np.reshape(scale_minmax(cv2.filter2D(img, -1, sfilter)),(125*125, 1))
            
            if key not in texton_dict.keys():
                response_dict[key] = output
            else:
                response_dict[key] = np.append(texton_dict[key],output)

# 
responses = np.zeros((img.shape[0]*img.shape[1]*sample_size*5,len(sparams)))

keys = list(texton_dict.keys())
for col, key in enumerate(keys):
    responses[::,col] = texton_dict[key]


In [88]:
import pickle
kmeans_min = MiniBatchKMeans(n_clusters=30, random_state=0).fit(responses)
with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([kmeans_min], f)

In [89]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=30, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [90]:
texon_dict = {}

for point in responses: 
    for cluster in kmeans.cluster_centers_:
        



array([[0.84505033, 0.87849689, 0.51630596, 0.8965107 , 0.5169073 ,
        0.4880106 , 0.90847753, 0.51580634, 0.47980082, 0.91599786,
        0.5154197 , 0.47887554, 0.48596015],
       [0.16080009, 0.13195782, 0.32487698, 0.11753003, 0.27300028,
        0.46535387, 0.11383696, 0.2519885 , 0.47817644, 0.11327448,
        0.24155839, 0.47318858, 0.51321558],
       [0.64598131, 0.66762715, 0.50573874, 0.68380049, 0.50583412,
        0.68333425, 0.69952466, 0.5068038 , 0.70469007, 0.71286302,
        0.50732312, 0.7104864 , 0.49142604],
       [0.47548103, 0.47157217, 0.58979144, 0.48025358, 0.61830188,
        0.31519721, 0.49215436, 0.62923456, 0.29157163, 0.50186515,
        0.63765867, 0.28864811, 0.49893385],
       [0.47368313, 0.47025714, 0.40256364, 0.47906066, 0.37516428,
        0.33360647, 0.49073968, 0.35981184, 0.3120573 , 0.50016834,
        0.35466328, 0.30923412, 0.49960017],
       [0.1815231 , 0.1568551 , 0.58330235, 0.14439893, 0.60097676,
        0.66802238, 0.14300

In [ ]:
imgpath = "C:\\Users\peter\Documents\Cloud Project\Data\swimcat\B-pattern\images\\B_1img.png"
a = gabor_fn(1,0,1,0,1)
img = scale_minmax(cv2.imread(imgpath,1))
test = scale_minmax(cv2.filter2D(img, -1, a))
plt.figure(1)
plt.imshow(test)
plt.figure(2)
plt.imshow(a)

In [93]:
responses.shape

(390625, 13)